# Spaceship Adaboost Submission

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Bring the Data In

In [ ]:
X_df = pd.read_csv('spaceship_train_X_v2.csv')
y_df = pd.read_csv('spaceship_train_y.csv')
X_submission = pd.read_csv('spaceship_test_X_v2.csv')

y_df.drop('Unnamed: 0', axis=1, inplace=True)
X_df.head()

,PassengerId,CryoSleep,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Expenditure,NoSpending,...,CabinSide_Z,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,AgeEncoded
0,0001_01,0,0,-0.337530,-0.283865,-0.287383,-0.273826,-0.265831,-0.518357,1,...,0,0,1,0,0,0,0,0,0,3.0
1,0002_01,0,0,-0.172455,-0.278226,-0.245562,0.213965,-0.227033,-0.256582,0,...,0,0,0,0,0,0,1,0,0,2.0
2,0003_01,0,1,-0.272409,1.956643,-0.287383,5.692512,-0.222625,3.174596,0,...,0,1,0,0,0,0,0,0,0,4.0
3,0003_02,0,0,-0.337530,0.519986,0.333240,2.684020,-0.095651,1.322607,0,...,0,1,0,0,0,0,0,0,0,2.0
4,0004_01,0,0,0.121349,-0.240007,-0.034784,0.228181,-0.264067,-0.130318,0,...,0,0,0,0,0,0,1,0,0,1.0


In [ ]:
X_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4277 non-null   int64  
 2   VIP                        4277 non-null   int64  
 3   RoomService                4277 non-null   float64
 4   FoodCourt                  4277 non-null   float64
 5   ShoppingMall               4277 non-null   float64
 6   Spa                        4277 non-null   float64
 7   VRDeck                     4277 non-null   float64
 8   Expenditure                4277 non-null   float64
 9   NoSpending                 4277 non-null   int64  
 10  Group                      4277 non-null   int64  
 11  GroupSize                  4277 non-null   int64  
 12  Solo                       4277 non-null   int64  
 13  CabinRegion_1              4277 non-null   int64

In [ ]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8693 non-null   int64  
 2   VIP                        8693 non-null   int64  
 3   RoomService                8693 non-null   float64
 4   FoodCourt                  8693 non-null   float64
 5   ShoppingMall               8693 non-null   float64
 6   Spa                        8693 non-null   float64
 7   VRDeck                     8693 non-null   float64
 8   Expenditure                8693 non-null   float64
 9   NoSpending                 8693 non-null   int64  
 10  Group                      8693 non-null   int64  
 11  GroupSize                  8693 non-null   int64  
 12  Solo                       8693 non-null   int64  
 13  CabinRegion_1              8693 non-null   int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
# Mengubah target menjadi 1D array
y_train = y_train['Transported'].values
y_test = y_test['Transported'].values


# Base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)

# Membuat pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalisasi data
    ('classifier', AdaBoostClassifier(base_estimator=base_estimator))
])

parameters = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.1, 1]
}

# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
X_train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 6954 entries, 2333 to 7270
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                6954 non-null   object 
 1   CryoSleep                  6954 non-null   int64  
 2   VIP                        6954 non-null   int64  
 3   RoomService                6954 non-null   float64
 4   FoodCourt                  6954 non-null   float64
 5   ShoppingMall               6954 non-null   float64
 6   Spa                        6954 non-null   float64
 7   VRDeck                     6954 non-null   float64
 8   Expenditure                6954 non-null   float64
 9   NoSpending                 6954 non-null   int64  
 10  Group                      6954 non-null   int64  
 11  GroupSize                  6954 non-null   int64  
 12  Solo                       6954 non-null   int64  
 13  CabinRegion_1              6954 non-null   int64  

# Model Training

In [ ]:
# Membuat GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=cv, scoring='accuracy', verbose=1)
grid_search.fit(X_train, y_train)

#Evaluasi model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 9 candidates, totalling 90 fits
Best parameters: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 200}
Accuracy on test set: 0.7860839562967222


In [ ]:
# Evaluasi pada validation set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.76      0.78       861
           1       0.77      0.82      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739

Confusion Matrix:
[[651 210]
 [162 716]]


In [ ]:
best_ada = grid_search.best_estimator_
y_pred_light = best_ada.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_light))
print("Classification Report:\n", classification_report(y_test, y_pred_light))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_light))

Best parameters: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 200}
Accuracy: 0.7860839562967222
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78       861
           1       0.77      0.82      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739

Confusion Matrix:
 [[651 210]
 [162 716]]


In [ ]:
# Melakukan cross-validation dengan GridSearchCV
scores = cross_val_score(best_ada, X_df, y_df, cv=5)

# Menampilkan hasil cross-validation
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

In [ ]:
y_pred = grid_search.predict(X_test)
print("Best parameters:", grid_search.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Best parameters: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 200}
Accuracy: 0.7860839562967222
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.76      0.78       861
           1       0.77      0.82      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739

Confusion Matrix:
 [[651 210]
 [162 716]]


# Submission Prediction

In [ ]:
# For training, we use ALL data from spaceship_train_X_v2.csv and spaceship_train_y.csv
grid_search.fit(X_df, y_df['Transported'].values)
# Prediksi data submission
y_prediction = grid_search.predict(X_submission)
print(y_prediction)


Fitting 10 folds for each of 9 candidates, totalling 90 fits
[1 0 1 ... 1 1 1]


In [ ]:
submission_dict = {'PassengerId':X_submission['PassengerId'], 'Transported':y_prediction.astype('bool')}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


## Export CSV

In [ ]:
submission_dict.to_csv('spaceship_adaboost3.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>